In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r"bank-full.csv", sep=";")
df.sample()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
26170,30,technician,divorced,secondary,no,3100,yes,no,cellular,20,nov,125,1,183,4,failure,no


In [3]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [4]:
data = df[['age', 'job', 'marital', 'education', 'balance', 'housing',
        'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y']]

In [5]:
question = """
Question 1
What is the most frequent observation (mode) for the column education?

unknown
primary
secondary
tertiary
"""
print(question)

data['education'].value_counts()


Question 1
What is the most frequent observation (mode) for the column education?

unknown
primary
secondary
tertiary



,count
education,
secondary,23202
tertiary,13301
primary,6851
unknown,1857


In [6]:
question = """
Question 2
Create the correlation matrix for the numerical features of your dataset.
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

age and balance
day and campaign
day and pdays
pdays and previous
"""
print(question)
corr_one = data['age'].corr(data['balance'], method='pearson')
corr_two = data['day'].corr(data['campaign'], method='pearson')
corr_three = data['day'].corr(data['pdays'], method='pearson')
corr_four = data['pdays'].corr(data['previous'], method='pearson')
print("Correlation:", corr_one, corr_two, corr_three, corr_four)


Question 2
Create the correlation matrix for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

age and balance
day and campaign
day and pdays
pdays and previous

Correlation: 0.09778273937134754 0.16249021632619293 -0.09304407377294052 0.45481963548050075


In [7]:
question = """
Question 3

Calculate the mutual information score between y and
other categorical variables in the dataset. Use the training set only.

Round the scores to 2 decimals using round(score, 2).
Which of these variables has the biggest mutual information score?

contact
education
housing
poutcome
"""
print(question)


Question 3

Calculate the mutual information score between y and 
other categorical variables in the dataset. Use the training set only.

Round the scores to 2 decimals using round(score, 2).
Which of these variables has the biggest mutual information score?

contact
education
housing
poutcome



In [8]:
from sklearn.model_selection import train_test_split

df['y'] = df['y'].map({'no':0, 'yes':1})

categorical = data.dtypes[data.dtypes=='object'].index

df_train_full, df_test = train_test_split(data, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_train_full, test_size=0.25, random_state=11)

In [9]:
from sklearn.metrics import mutual_info_score, accuracy_score

def calculate_mi(series):
    return mutual_info_score(series, df_train.y)

df_mi = df_train[categorical].apply(calculate_mi)
df_mi = df_mi.sort_values(ascending=False).to_frame(name='MI')


display(df_mi)

,MI
y,0.360402
poutcome,0.028691
month,0.026111
contact,0.013213
housing,0.009586
job,0.009228
education,0.002442
marital,0.001778


In [10]:
question = """
Question 4
Now let's train a logistic regression.
Remember that we have several categorical variables in the dataset.
Include them using one-hot encoding.
Fit the model on the training dataset.
To make sure the results are reproducible across different versions of Scikit-Learn,
fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
What accuracy did you get?
"""
print(question)


Question 4
Now let's train a logistic regression.
Remember that we have several categorical variables in the dataset.
Include them using one-hot encoding.
Fit the model on the training dataset.
To make sure the results are reproducible across different versions of Scikit-Learn, 
fit the model with these parameters:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
Calculate the accuracy on the validation dataset and round it to 2 decimal digits.
What accuracy did you get?



In [11]:
from sklearn.feature_extraction import DictVectorizer
dv = DictVectorizer(sparse=False)

X = data.drop('y', axis=1)
y = data.y
X = X.to_dict(orient='records')

x_train_full, x_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.25, random_state=11)

x_train= dv.fit_transform(x_train)
x_val  = dv.transform(x_val)
x_test = dv.transform(x_test)

In [12]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(x_train,y_train)
y_val_pred = model.predict(x_val)

acc_score = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy is {acc_score:.2f}")

Validation accuracy is 0.90


In [13]:
question = """
Question 5

Let's find the least useful feature using the feature elimination technique.
Train a model with all these features (using the same parameters as in Q4).
Now exclude each feature from this set and train a model without it.
Record the accuracy for each model.

For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
Which of following feature has the smallest difference?

age
balance
marital
previous
Note: The difference doesn't have to be positive.
"""

print(question)


Question 5

Let's find the least useful feature using the feature elimination technique.
Train a model with all these features (using the same parameters as in Q4).
Now exclude each feature from this set and train a model without it. 
Record the accuracy for each model.

For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
Which of following feature has the smallest difference?

age
balance
marital
previous
Note: The difference doesn't have to be positive.



In [14]:
X = data.drop(['y','age'], axis=1)
y = data.y
X = X.to_dict(orient='records')

x_train_full, x_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.25, random_state=11)

x_train= dv.fit_transform(x_train)
x_val  = dv.transform(x_val)
x_test = dv.transform(x_test)

age_model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
age_model.fit(x_train,y_train)
y_val_pred = age_model.predict(x_val)

acc_score = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy without age is {acc_score:.2f}")

Validation accuracy without age is 0.90


In [15]:
X = data.drop(['y','balance'], axis=1)
y = data.y
X = X.to_dict(orient='records')

x_train_full, x_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.25, random_state=11)

x_train= dv.fit_transform(x_train)
x_val  = dv.transform(x_val)
x_test = dv.transform(x_test)

balance_model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
balance_model.fit(x_train,y_train)
y_val_pred = balance_model.predict(x_val)

acc_score = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy without balance is {acc_score:.2f}")

Validation accuracy without balance is 0.90


In [16]:
X = data.drop(['y','marital'], axis=1)
y = data.y
X = X.to_dict(orient='records')

x_train_full, x_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.25, random_state=11)

x_train= dv.fit_transform(x_train)
x_val  = dv.transform(x_val)
x_test = dv.transform(x_test)

marital_model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
marital_model.fit(x_train,y_train)
y_val_pred = marital_model.predict(x_val)

acc_score = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy without marital is {acc_score:.2f}")

Validation accuracy without marital is 0.90


In [17]:
X = data.drop(['y','previous'], axis=1)
y = data.y
X = X.to_dict(orient='records')

x_train_full, x_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
x_train, x_val, y_train, y_val = train_test_split(x_train_full, y_train_full, test_size=0.25, random_state=11)

x_train= dv.fit_transform(x_train)
x_val  = dv.transform(x_val)
x_test = dv.transform(x_test)

previous_model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
previous_model.fit(x_train,y_train)
y_val_pred = previous_model.predict(x_val)

acc_score = accuracy_score(y_val, y_val_pred)
print(f"Validation accuracy without previous is {acc_score:.2f}")

Validation accuracy without previous is 0.90


In [18]:
question = """
Question 6

Now let's train a regularized logistic regression.
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
Train models using all the features as in Q4.

Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
Which of these C leads to the best accuracy on the validation set?

0.01
0.1
1
10
100
Note: If there are multiple options, select the smallest C
"""

print(question)


Question 6

Now let's train a regularized logistic regression.
Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
Train models using all the features as in Q4.

Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
Which of these C leads to the best accuracy on the validation set?

0.01
0.1
1
10
100
Note: If there are multiple options, select the smallest C



In [19]:
C_values = [0.01, 0.1, 1, 10, 100]    # Values of C to try
accuracy_dict = {}

for C in C_values:
    model = LogisticRegression(C=C, solver='liblinear', max_iter=1000)
    model.fit(x_train, y_train)

    # Predict on validation set
    y_pred = model.predict(x_val)

    # Calculate accuracy and round to 3 decimal places
    accuracy = round(accuracy_score(y_val, y_pred), 4)

    # Store the accuracy for each C
    accuracy_dict[C] = accuracy

print(f"Accuracy for each C:", accuracy_dict)

Accuracy for each C: {0.01: 0.8953, 0.1: 0.8964, 1: 0.897, 10: 0.8969, 100: 0.8973}
